<h1 style="text-align:center;line-height:1.5em;font-size:30px;">Transfer Learning with a Convolutional Neural Network for Hydrological Streamline Detection</h1>

<p style="text-align:center;">
    Nattapon Jaroenchai$^{a, b}$ Shaohua Wang$^{a, b}$, Li Chen$^{a, b}$, Lawrence V. Stanislawski$^{c}$, Ethan Shavers$^{c}$, E. Lynn Usery$^{c}$, Shaowen Wang$^{a, b}$
</p>

<p style="text-align:center;">
$^{a}$ Department of Geography and Geographic Information Science, University of Illinois at Urbana-Champaign, Urbana, IL, US<br>
$^{b}$ CyberGIS Center for Advanced Digital and Spatial Studies, University of Illinois at Urbana-Champaign, Urbana, IL, USA<br>
$^{c}$ U.S. Geology Survey, Center of Excellence for Geospatial Information Science, Rolla, MO, USA <br>
$^{d}$ School of Geoscience and Info-Physics, Central South University, Changsha, Hunan, China <br>
</p>

---
    
**Notebook Structure:**
- [Introduction](introduction.ipynb)
- Codes
 - [Data Preprocessing](preprocessing.ipynb)
 - [Experiment 1: different input datasets](experiment_1.ipynb)
 - [Experiment 2: retrain different part of the network](experiment_2.ipynb)
 - [Experiment 3: different sample sizes](experiment_3.ipynb)
 - [Model Evaluation](evaluation.ipynb) 
 - [Conclusion](conclusion.ipynb) 

### Load libraries

In this step we use Pillow libraries to read raster images and then stack the images into data cube using NumP. Then we define the array of images's file name. 

In [2]:
from PIL import Image as im
import numpy as np
import glob
im.MAX_IMAGE_PIXELS = None

### Define array of input files
We also define the range of max and min pixel value of each raster image. The range are calculated using the range of 3 standard diveations or 99.7% of all pixels. This is to prevent the problem in normalization process.

### Choose which data set we want to create

In this study we use 2 datasets:
1. The dataset the only cantains DEM derived data 
2. The dataset with NAIP bands raster images 

### If you want to create data wihtout NAIP run the code block below

In [3]:
# only DEM derived data
files = files = [
    '01_CovingtonRiver_020801030302_CURVATURE_nodata.tif',
    '02_CovingtonRiver_020801030302_SLOPE_nodata.tif', 
    '03_CovingtonRiver_020801030302_OPENESS_nodata.tif',
    '04_CovingtonRiver_020801030302_DEM_nodata.tif', 
    '05_CovingtonRiver_020801030302_TPI_21_nodata.tif', 
    '06_CovingtonRiver_020801030302_INTENSITY_nodata.tif',
    '07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif',  
    '08_TPI_CovingtonRiver_020801030302_3_nodata.tif',
    ]
#The ranges here are the range of 3SD or 99.7% of all data
#This is to prevent problem in the normalization process
ranges =[
    [-0.28222607489286866,0.28248185206077936], # '01_CovingtonRiver_020801030302_CURVATURE_nodata.tif'
    [-0.28097646432537005,0.7456170822778501], # '02_CovingtonRiver_020801030302_SLOPE_nodata.tif'
    [81.4260223726079, 96.2421904790021], # '03_CovingtonRiver_020801030302_OPENESS_nodata.tif'
    [-207.05215950623995, 938.84507252262], # '04_CovingtonRiver_020801030302_DEM_nodata.tif'
    [-0.8616536368618285, 0.8618900909336314], # '05_CovingtonRiver_020801030302_TPI_21_nodata.tif'
    [1.494045913614002, 77.145486958992],# '06_CovingtonRiver_020801030302_INTENSITY_nodata.tif'
    [3.1338925783175906, 8.81583142566701], # '07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif'
    [-0.19840276259099499, 0.198406762590995] # '08_TPI_CovingtonRiver_020801030302_3_nodata.tif'
    ]
name_suffix = "without_NAIP"

### In you want to generate the dataset with NAIP iamgery, run the code block below

In the dataset with NAIP raster iamges, we substitute slope, openness, LiDAR intensity, and TPI with window size 3 with red, green, blue, and near infrared bands of NAIP imagery.

In [ ]:
# # Files' name of all the ionput data
# files = [
#     '01_CovingtonRiver_020801030302_CURVATURE_nodata.tif',
#     '04_CovingtonRiver_020801030302_DEM_nodata.tif', 
#     '05_CovingtonRiver_020801030302_TPI_21_nodata.tif', 
#     '07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif',  
#     'NAIP/NAIP2018_RED_edit_nodata.tif',
#     'NAIP/NAIP2018_GREEN_edit_nodata.tif',
#     'NAIP/NAIP2018_BLUE_edit_nodata.tif',
#     'NAIP/NAIP2018_INFARED_edit_nodata.tif' 
#     ]

# #The ranges here are the range of 3SD or 99.7% of all data
# #This is to prevent problem in the normalization process
# ranges =[
#     [-0.28222607489286866,0.28248185206077936], # '01_CovingtonRiver_020801030302_CURVATURE_nodata.tif'
#     [-207.05215950623995, 938.84507252262], # '04_CovingtonRiver_020801030302_DEM_nodata.tif'
#     [-0.8616536368618285, 0.8618900909336314], # '05_CovingtonRiver_020801030302_TPI_21_nodata.tif'
#     [3.1338925783175906, 8.81583142566701], # '07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif'
#     ]
# name_suffix = "with_NAIP"

 ### Define normalization funcition
 
 We normalize pixel values using the equation below. 
 
<img src="img/preprocess_normalize_equation.png">


In [4]:
# Normalize function
def normalize(array_x, min_x, max_x):
    
    # set all pixels to be within the range.
    array_x[ (array_x!=-9999) & (array_x < min_x) ] = min_x
    array_x[ (array_x!=-9999) & (array_x > max_x) ] = max_x
    
    # normalize array 
    array_x[array_x!=-9999] = ((array_x[array_x!=-9999]-min_x)/(max_x-min_x))*255
    
    return array_x

### Create the total dataset of the whole study area

Note: We cannot run the preprocessing process due to the memory limit on CyberGISX 

In [ ]:
# folder contains the TIFF files
data_folder = './organized_data/raw/'

#initialize the output
output = [] 
output = np.array(output)

for num, file in enumerate(files, start = 0 ): 
    
    path = data_folder+file
    print(path)

    image = im.open(path)
    image_array = np.array(image)
    print(image_array.shape)
    
    # the first image is used to initialize the size of the output array
    if (num == 0):
        output=np.empty((len(files),image_array.shape[0], image_array.shape[1]))
        
    # The null value in "07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif" has to be set to -9999 instead of 255. Then normalize the raster. 
    if(files == '07_CovingtonRiver_020801030302_Cov10cell_Geomorphon.tif'):
        image_array[image_array==255] = -9999
        image_array = normalize(image_array,ranges[num][0],ranges[num][1])
        
    # For NAIP bands image we have to set null to -9999 instead of 0. We do not normalize NAIP images because all data are normalized to 0-255 which is the range of NAIP dataset. 
    elif("NAIP" in file):
        image_array[image_array==0] = -9999
    
    # Normalize the image.
    else:
        image_array = normalize(image_array,ranges[num][0],ranges[num][1])

    print("Normalized ", path)
    
    # Each raster images are stacked in out put array. 
    output[num] = image_array

# shift the shape of output array to (width, height, img#) 
output = np.moveaxis(output,0,-1)

# Save the array as NPY file, named total_without_NAIP.
np.save('total_'+name_suffix,output)
print('saved total')

./organized_data/raw/01_CovingtonRiver_020801030302_CURVATURE_nodata.tif
(13867, 14406)
Normalized  ./organized_data/raw/01_CovingtonRiver_020801030302_CURVATURE_nodata.tif
./organized_data/raw/02_CovingtonRiver_020801030302_SLOPE_nodata.tif
(13867, 14406)
Normalized  ./organized_data/raw/02_CovingtonRiver_020801030302_SLOPE_nodata.tif
./organized_data/raw/03_CovingtonRiver_020801030302_OPENESS_nodata.tif
(13867, 14406)
Normalized  ./organized_data/raw/03_CovingtonRiver_020801030302_OPENESS_nodata.tif
./organized_data/raw/04_CovingtonRiver_020801030302_DEM_nodata.tif
(13867, 14406)
Normalized  ./organized_data/raw/04_CovingtonRiver_020801030302_DEM_nodata.tif
./organized_data/raw/05_CovingtonRiver_020801030302_TPI_21_nodata.tif
(13867, 14406)
Normalized  ./organized_data/raw/05_CovingtonRiver_020801030302_TPI_21_nodata.tif
./organized_data/raw/06_CovingtonRiver_020801030302_INTENSITY_nodata.tif


### Create study area mask

We create the mask of the watershed using the curvature raster image. The pixels outside of the study area are set to 0 and pixels inside are set to 1. 

In [3]:
def generate_mask():
    
    path = './organized_data/raw/01_CovingtonRiver_020801030302_CURVATURE_nodata.tif'

    image = im.open(path)
    image_array = np.array(image)
    
    # set pixels out side to 0 and inside to 1
    mask = image_array != -9999
    
    # the pixel values must be converted to int to make sure that we can use it as boolean datatype.
    mask.astype(int)
    
    # save the mask array as mask.npy
    np.save('mask',mask)
    print('save mask')

generate_mask()

save mask


### Create reference dataset

For the reference data we read the raster image and directly convert it to Numpy array. 

In [5]:
def generate_reference():
    
    # read the reference TIFF file 
    path = './organized_data/raw/reference_nodata.tif'
    image = im.open(path)
    
    # canvert the raster to Numpy array
    image_array = np.array(image)    
    
    # set the null value (-9999) to NumPy NaN value
    image_array[image_array==-9999] = 0
    
    # save the array to reference_as_None.npy
    np.save('reference',image_array)
    print('saved reference')

generate_reference()

saved reference


<hr>

# Generate traning and validation sample patches

We divide the study area into two parts, top and bottom. The training and validation sample patches are generated from the top part of study area.   
The entire bottom part is used to generate testing sample patches using the moving window strategy with 30 pixels buffer. 


In [2]:
# install required libraries for preprocessing.
!pip install --user imgaug==0.4.0
!pip install --user intervaltree
!pip install --user numpy --upgrade
    
# Please restart the kernel after this block finished running.

  Using cached numpy-1.21.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
ERROR: libpysal 4.2.2 requires bs4, which is not installed.
ERROR: tensorflow 2.0.4 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.21.1 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 96.9 MB 50 kB/s s eta 0:00:01             | 1.4 MB 460 kB/s eta 0:03:28               | 2.7 MB 460 kB/s eta 0:03:25                   | 3.4 MB 460 kB/s eta 0:03:233 MB 460 kB/s eta 0:03:21ta 0:01:21                    | 14.6 MB 1.1 MB/s eta 0:01:16B 1.1 MB/s eta 0:01:16  | 17.0 MB 1.1 MB/s eta 0:01:141 MB/s eta 0:01:13██▋                         | 19.9 MB 1.1 MB/s eta 0:01:11            | 21.0 MB 1.2 MB/s eta 0:01:03 MB 1.2 MB/s eta 0:01:02 MB 1.2 MB/s eta 0:01:01��████▋                       | 26.0 MB 1.2 MB/s eta 0:00:59 MB 1.2 MB/s eta 0:00:58��████▍              

### Load libraries

In this step we use intervaltree library for checking the overlap between patches.

In [7]:
import copy
import random
import sys
from intervaltree import Interval, IntervalTree

### Define functions for checking if the patch is valid

The condition to be valid are:

1. the patch size is 224 pixels by 224 pixels and located within the study area. 
2. The patch of validation must not overlap with training patches. 

In [9]:
# Define function for checking if the patch is valid 
def is_invalid(mode, temp, templ, minrow, maxrow, mincolumn, maxcolumn):

    # check complete patch
    is_not_complete_patch = np.any(temp[0,:,:,-1] <= 0) or temp.shape[1:3] != (patch_size,patch_size) or templ.shape[1:] != (patch_size,patch_size)

    if mode =="training":
        # training sample can overlap with itself 
        # so, we only check if it is complete patch or not
        return is_not_complete_patch

    if mode =="validation":
        # validation sample **cannot** overlap with training samples patches
        # so, we need to 1) check the completeness 2) check if it overleps with training samples

        is_overlap = False

        row_overlap = samples_row[minrow:maxrow]
        column_overlap = samples_column[mincolumn:maxcolumn]

        for row_interval in row_overlap:
            begin, end, row_data = row_interval
            for column_interval in column_overlap:
                begin, end, column_data = column_interval
                if(row_data == column_data):
                    is_overlap = True
                    # print(minrow, maxrow, mincolumn, maxcolumn)
                    # print(row_interval, column_interval)
                    break
            else:
                continue
            break

        return bool(is_not_complete_patch or is_overlap)



Extracting data patches!


### Define function fro sample patches extraction

After we have the valid location of all sample patches, we extract the data from the total dataset. The function returns a tuple of array of all sample patches and their labels. 

In [ ]:
#extract stream/non-stream samples using the random patches
def generate_samples(totaldata,row,col,label,size,train_or_vali):
    
    global start_index
    global samples_row
    global samples_column
    global training_sample
    global validation_sample

    count = 0
    row = row[start_index:]
    col = col[start_index:]

    for index,(i,j) in enumerate(zip(row,col)):

        # calculate min max ranges of the patch.
        minrow = (i-int(patch_size/2))
        maxrow = (i+int(patch_size/2))
        mincolumn = (j-int(patch_size/2))
        maxcolumn = (j+int(patch_size/2))

        # extract data from total dataset and label of total dataset
        temp = totaldata[minrow:maxrow,mincolumn:maxcolumn,:][np.newaxis,:,:,:]
        templ = label[minrow:maxrow,mincolumn:maxcolumn][np.newaxis,:,:]

        # validate the conditions
        if is_invalid(train_or_vali,temp,templ,minrow,maxrow,mincolumn,maxcolumn):
            # if not complete (or overlap in validation) skip this patch.
            continue 
        else:
            # if valid add to samples
            count += 1
            if count == 1:            
                train_vali = temp[:,:,:,:-1]
                train_vali_label = templ
            else:
                train_vali = np.concatenate((train_vali, temp[:,:,:,:-1]),axis = 0)
                train_vali_label = np.concatenate((train_vali_label, templ),axis = 0)
            
            if train_or_vali == "training":
                training_sample.append((minrow,mincolumn))

                # if it is training sample, add the sample patch x and y ranges to the trees with the count as the ranges' label.
                samples_row[minrow:maxrow] = count
                samples_column[mincolumn:maxcolumn] = count

            if train_or_vali == "validation":
                validation_sample.append((minrow,mincolumn))

        if count == size:
            print("Generated "+str(train_or_vali)+": "+str(count)+" samples and used from "+str(start_index)+" to "+str(start_index+index)+"random rows and columns")
            start_index = start_index+index
            return [train_vali,train_vali_label]
    
    # If the algorithm cannot find enough sample patches, print the message say not enough samples. 
    print("Not enough random smaples")
    print("Generated "+str(train_or_vali)+": "+str(count)+" samples out of "+ str(size)+ " end at index "+str(index))
    return 

print('Extracting data patches!')

### The extraction process 

At this step we will actually run the whole process. 

1. Load dataset: the total dataset, mask, and reference datasets are loaded from NPY files. 
2. Randomize the sample locations: 100,000 pixels of stream and non stram pixels are randomly selected from the reference. Their location (row and columns) are kept in separated arrays. 
3. Generate the samples: from the randomly selected pixels, each of them are evaluated if it is valid by the function above. Then, all valid pixels are saved to NPY files.

In [8]:
####### Generate training/validation/testing image patches 
#Total data dimension: 14406*13867

patch_size = 224 #patch size of each sample

###### 1. Load datasets
# read the total dataset 
totaldata = np.load('total_'+name_suffix+'.npy')

# read the mask 
mask = np.load('mask.npy')

# add mas to the total data array for convinience
totaldata = np.concatenate((totaldata,mask[:,:,np.newaxis]),axis = 2)]

# load the reference data
label = np.load('reference.npy')

print('Completed: Data Loading!')

###### 2. Prepare the location of the stream and non stream patches by random sample from the reference raster
# Use the upper half to generate training and validation data
half = totaldata.shape[0]//2
label_train_vali = label[:half]
[r,c] = np.where(label_train_vali == True) #steamline patches
[rn,cn] = np.where(label_train_vali == False) #non-steamline patches

# randomly select 100000 sample from stream and non-stream pixels. 
inder = random.sample(range(0, len(r)-1), 100000)
indenr = random.sample(range(0, len(rn)-1), 100000)

# get the row and column value of each stream and non-stream pixel 
r,c = r[inder],c[inder]
rn,cn = rn[indenr],cn[indenr]

print('Get random coordinates of stream and non-stream pixels!')

###### 3. generate the samples 
# Initialize the output array and tree structure for search algorithm
samples_row = IntervalTree()
samples_column = IntervalTree()
validation_sample = []
training_sample = []

start_index = 0

# Generate training smaples 300 patches for both stream and non-stream
training_samples_num = 300
[train_vali_stream,train_vali_stream_label] = generate_samples(totaldata,r,c,label_train_vali,training_samples_num,"training")
[train_vali_nonstream,train_vali_nonstream_label] = generate_samples(totaldata,rn,cn,label_train_vali,training_samples_num,"training")
trainvali_data = np.concatenate((train_vali_stream,train_vali_nonstream),axis = 0)
trainvali_label = np.concatenate((train_vali_stream_label,train_vali_nonstream_label),axis = 0)

# Shuffle training and validation samples
s = np.arange(trainvali_data.shape[0])
np.random.shuffle(s)
train_data = trainvali_data[s]
train_label = trainvali_label[s]

#Save the trainging samples both data and label
np.save('./samples/train_data_'+name_suffix+'.npy',train_data)
np.save('./samples/train_label_'+name_suffix+'.npy',train_label[:,:,:,np.newaxis])
training_sample = np.array(training_sample)
np.save('./samples/train_patches_top-left_'+name_suffix+'.npy',training_sample)

# Generate validation smaples 300 patches for both stream and non-stream
validation_samples_num = 300
[train_vali_stream,train_vali_stream_label] = generate_samples(totaldata,r,c,label_train_vali,validation_samples_num,"validation")
[train_vali_nonstream,train_vali_nonstream_label] = generate_samples(totaldata,rn,cn,label_train_vali,validation_samples_num,"validation")
trainvali_data = np.concatenate((train_vali_stream,train_vali_nonstream),axis = 0)
trainvali_label = np.concatenate((train_vali_stream_label,train_vali_nonstream_label),axis = 0)

# Shuffle training and validation samples
s = np.arange(trainvali_data.shape[0])
np.random.shuffle(s)
vali_data = trainvali_data[s]
vali_label = trainvali_label[s]

#Save the validation samples both data and label
np.save('./samples/vali_data_'+name_suffix+'.npy',vali_data)
np.save('./samples/vali_label_'+name_suffix+'.npy',vali_label[:,:,:,np.newaxis])
validation_sample = np.array(validation_sample)
np.save('./samples/vali_patches_top-left_'+name_suffix+'.npy',validation_sample)

FileNotFoundError: [Errno 2] No such file or directory: './Total_data/total_without_NAIP.npy'

# Generate Test samples

The test sample patches are generated using moving window with 30 pixel buffer. 

In [ ]:
import os
import numpy as np
import copy
import random
from datetime import datetime

# stream/non-stream sample size
patch_size = 224 #patch size of each sample

#Total data dimension: 13927, 14466
totaldata = np.load('total_'+name_suffix+'.npy')
mask = np.load('mask.npy')
#Add mask 
totaldata = np.concatenate((totaldata,mask[:,:,np.newaxis]),axis = 2)

label = np.load('reference.npy')

print('Completed: Data Loading!')

# buffer size
buf = 30
it = 'full'
# Image dimension
IMG_WIDTH = 224
IMG_HEIGHT = 224

# moving window size = image_dimension - 2*buffer_size
mw = IMG_WIDTH - buf*2

half = totaldata.shape[0]//2
bottom_half_total = totaldata[half:,:,:]

bottom_half_mask = mask[half:,:]
print("mask",bottom_half_mask.shape)
np.save("./train_test_dataset/bottom_half_test_mask.npy",bottom_half_mask)
print('Saved Mask!')

bottom_half_label = label[half:,:]
print("label",bottom_half_label.shape)
np.save("./train_test_dataset/bottom_half_test_label.npy",bottom_half_label)
print('Saved Label!')

# Number of trainig channels
# Adding padding to width and height for moving window 
totalnew = np.pad(bottom_half_total, ((buf, buf),(buf,buf),(0,0)), 'symmetric')

#The last dimension is the mask 
#totalnew = totalnew[:,:,buf:(buf+9)]
totalnew = totalnew[:,:,(0,1,2,3,4,5,6,7)]
print(totalnew.shape)

#get taotal data height and width
dim = bottom_half_total.shape[:2]

# number of patch rows
numr = dim[0]//(IMG_WIDTH - buf*2)#224
print('rows:'+str(numr))

# number of patch columns
numc = dim[1]//(IMG_WIDTH - buf*2)#224
# only left side
# numc = dim[1]//2//(IMG_WIDTH - buf*2)#224
print('columns:'+str(numc))

# Splitting the total data into patches 4
count = 0
for i in range(numr):
    print("row: ",i)
    for j in range(numr):
        # print("column: ",j)
        count += 1
        temp = totalnew[i*mw:(i*mw+224),j*mw:(j*mw+224),:][np.newaxis,:,:,:]
        if count == 1:
            total = temp
        else:
            total = np.concatenate((total, temp),axis = 0)
        
print(total.shape)
# Save the total dataset
np.save("./train_test_dataset/bottom_half_test_data.npy",total)
print("Testing moving window is generate!")
